# Read data

In [1]:
import pandas as pd
import os

In [2]:
path = "data/AIEP_common_user_questions_India_Version.tsv"
basename = os.path.basename(path)
basename = os.path.splitext(basename)[0]

qa_df = pd.read_csv(path, sep="\t", header=0)
qa_df.describe(include="all")

,Question,Correct response,Follow-up questions needed,Comment
count,42,42,0.0,1
unique,42,41,NaN,1
top,What factors affect the price of Guntur Chilli?,"Dear Farmer, the following are the best onion ...",NaN,"Source: Basic Animal Husbandry Statistics, MoF..."
freq,1,2,NaN,1
mean,NaN,NaN,NaN,NaN
std,NaN,NaN,NaN,NaN
min,NaN,NaN,NaN,NaN
25%,NaN,NaN,NaN,NaN
50%,NaN,NaN,NaN,NaN
75%,NaN,NaN,NaN,NaN


In [3]:
qa_df = qa_df.rename(columns={"Question": "question"})

qa_df['question_id'] = qa_df.index + 1

In [4]:
qa_df

,question,Correct response,Follow-up questions needed,Comment,question_id
0,What factors affect the price of Guntur Chilli?,"Dear Farmer, Following are the major reasons t...",NaN,NaN,1
1,What is the appropriate age for vaccination of...,"Dear Farmer friend, following are the differen...",NaN,NaN,2
2,What cheyyali to prevent the mango tree from r...,"Dear Farmer, Fruit rotting is a fungal disease...",NaN,NaN,3
3,What are alternative crops to cowdung?,"Dear Farmer, The following green manure crops ...",NaN,NaN,4
4,What are the effects of marigold as a trap crop?,"Dear Valuable Farmer, Marigold as a trap crop ...",NaN,NaN,5
5,How do I prevent leaf blight in chili?,"Dear Farmer, to prevent leaf blight in chilli ...",NaN,NaN,6
6,Can the quantity of grain be increased in the ...,"Dear Farmer, No, the quantity of grain cannot...",NaN,NaN,7
7,How do I to recognize milk fever in cows?,"Dear Farmer, Milk fever is also known as hypoc...",NaN,NaN,8
8,Can animal feed be made at home?,"Dear Farmer, Yes, the animal feed can be made ...",NaN,NaN,9
9,How do I to choose a suitable breed of cow?,"Namasthe Farmer Friend, the following are the ...",NaN,NaN,10


# Set up API keys

In [5]:
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

True

In [6]:
def open_ended_prompt_contextual(question, country):
    return (f"You provide smallholder farmers in {country} with high quality agricultural advice and information. "
            f"Please respond to the following question like an agronomist or agricultural extension agent would respond "
            f"to a smallholder farmer. Be factually correct, precise and make the response understandable for a smallholder farmer. "
            f"Do not use more than 10 sentences.\n\nQuestion: {question}")

In [7]:
import litellm

[2025-06-19 23:39:24 - httpx:1025 - INFO] HTTP Request: GET https://raw.githubusercontent.com/BerriAI/litellm/main/model_prices_and_context_window.json "HTTP/1.1 200 OK"


In [8]:
# set callbacks
litellm.success_callback = ["langfuse"]
litellm.failure_callback = ["langfuse"]

In [9]:
country = "India"

In [10]:
def get_llm_answers(df, model, country, **kwargs):
    ids = []
    prompts = []
    answers = []

    for _, row in df.iterrows():
        question = row["question"]
        id = row["question_id"]
        prompt = open_ended_prompt_contextual(question, country)

        response = litellm.completion(
            model=model,
            messages=[
                {"role": "user", "content": prompt}
            ],
            **kwargs
        )

        # Extract the answer from the response
        answer = response["choices"][0]["message"]["content"]

        # Append the id, prompt, and answer to the lists
        ids.append(id)
        prompts.append(prompt)
        answers.append(answer)

    # Create a DataFrame from the lists
    results_df = pd.DataFrame({
        "question_id": ids,
        "prompt": prompts,
        "answer": answers
    })

    # Save the DataFrame to a CSV file
    model_name = model.replace("/", "-").replace(".", "-")
    csv_path = os.path.join("results", f"{basename}-{model_name}.csv")
    print(f"Saving results to {csv_path}")
    results_df.to_csv(csv_path, index=False)

    return results_df

# OpenAI

In [11]:
gpt_4o_df = get_llm_answers(qa_df, "openai/gpt-4o-2024-08-06", country)

Saving results to results/AIEP_common_user_questions_India_Version-openai-gpt-4o-2024-08-06.csv


In [12]:
gpt_4o_df

,question_id,prompt,answer
0,1,You provide smallholder farmers in India with ...,The price of Guntur Chilli can be influenced b...
1,2,You provide smallholder farmers in India with ...,Vaccination is crucial for maintaining the hea...
2,3,You provide smallholder farmers in India with ...,To prevent your mango tree from rotting and fa...
3,4,You provide smallholder farmers in India with ...,While cow dung is commonly used as a natural f...
4,5,You provide smallholder farmers in India with ...,Marigolds are often used as a trap crop to pro...
5,6,You provide smallholder farmers in India with ...,"To prevent leaf blight in your chili plants, i..."
6,7,You provide smallholder farmers in India with ...,"Yes, the quantity of grain can potentially be ..."
7,8,You provide smallholder farmers in India with ...,"Milk fever, also known as hypocalcemia, often ..."
8,9,You provide smallholder farmers in India with ...,"Yes, you can make animal feed at home using lo..."
9,10,You provide smallholder farmers in India with ...,"To choose a suitable breed of cow, first consi..."


In [13]:
o1_df = get_llm_answers(qa_df, "openai/o1-2024-12-17", country, max_completion_tokens=25000, reasoning_effort="medium")

Saving results to results/AIEP_common_user_questions_India_Version-openai-o1-2024-12-17.csv


In [14]:
o1_df

,question_id,prompt,answer
0,1,You provide smallholder farmers in India with ...,The price of Guntur Chilli mostly depends on s...
1,2,You provide smallholder farmers in India with ...,Calves should generally start receiving vaccin...
2,3,You provide smallholder farmers in India with ...,1. Ensure the soil around the mango tree drain...
3,4,You provide smallholder farmers in India with ...,You can use green manure crops like sunhemp or...
4,5,You provide smallholder farmers in India with ...,Marigold can help attract and trap pests like ...
5,6,You provide smallholder farmers in India with ...,Leaf blight in chili is typically caused by fu...
6,7,You provide smallholder farmers in India with ...,"Yes, it is possible to increase grain yield wi..."
7,8,You provide smallholder farmers in India with ...,Milk fever usually occurs in cows right after ...
8,9,You provide smallholder farmers in India with ...,"Yes, you can make animal feed at home by combi..."
9,10,You provide smallholder farmers in India with ...,"First, decide whether you want a cow primarily..."


In [15]:
o3_df = get_llm_answers(qa_df, "openai/o3-2025-04-16", country, max_completion_tokens=25000, reasoning_effort="medium")

Saving results to results/AIEP_common_user_questions_India_Version-openai-o3-2025-04-16.csv


In [16]:
o3_df

,question_id,prompt,answer
0,1,You provide smallholder farmers in India with ...,1. The first thing buyers look at is quality: ...
1,2,You provide smallholder farmers in India with ...,1. No vaccine is needed at birth; just be sure...
2,3,You provide smallholder farmers in India with ...,1. Keep rain-water from standing near the base...
3,4,You provide smallholder farmers in India with ...,Cow-dung is only one kind of organic manure; i...
4,5,You provide smallholder farmers in India with ...,Marigold works like a “sacrificial” plant that...
5,6,You provide smallholder farmers in India with ...,"1. Use certified, disease-free seed or resista..."
6,7,You provide smallholder farmers in India with ...,"No. Ruminant animals (cow, buffalo, goat) mus..."
7,8,You provide smallholder farmers in India with ...,1. Milk fever (low blood calcium) usually stri...
8,9,You provide smallholder farmers in India with ...,"Yes, you can prepare good-quality animal feed ..."
9,10,You provide smallholder farmers in India with ...,"1. First decide your main goal: high milk, dra..."


In [17]:
o4_mini_df = get_llm_answers(qa_df, "openai/o4-mini-2025-04-16", country, max_completion_tokens=25000, reasoning_effort="medium")

Saving results to results/AIEP_common_user_questions_India_Version-openai-o4-mini-2025-04-16.csv


In [18]:
o4_mini_df

,question_id,prompt,answer
0,1,You provide smallholder farmers in India with ...,1. Quality of Guntur chillies (bright deep-red...
1,2,You provide smallholder farmers in India with ...,1. Calves receive protective antibodies from t...
2,3,You provide smallholder farmers in India with ...,1. Field lo water stagnate avvakunda drainage ...
3,4,You provide smallholder farmers in India with ...,Here are some good organic alternatives to cow...
4,5,You provide smallholder farmers in India with ...,1. Marigold planted alongside or around your m...
5,6,You provide smallholder farmers in India with ...,Leaf blight in chili is caused by fungi like A...
6,7,You provide smallholder farmers in India with ...,"Yes, you can still increase grain yields even ..."
7,8,You provide smallholder farmers in India with ...,Milk fever usually appears in high-yielding co...
8,9,You provide smallholder farmers in India with ...,"Yes, you can prepare good-quality feed at home..."
9,10,You provide smallholder farmers in India with ...,"1. First decide your main goal: milk, draught ..."


# Anthropic

In [19]:
claude_3_5_df = get_llm_answers(qa_df, "anthropic/claude-3-5-sonnet-20241022", country)

Saving results to results/AIEP_common_user_questions_India_Version-anthropic-claude-3-5-sonnet-20241022.csv


In [20]:
claude_3_5_df

,question_id,prompt,answer
0,1,You provide smallholder farmers in India with ...,The price of Guntur chilli is influenced by se...
1,2,You provide smallholder farmers in India with ...,Namaste farmer brother/sister. For healthy cat...
2,3,You provide smallholder farmers in India with ...,"Namaste farmer, I understand you're concerned ..."
3,4,You provide smallholder farmers in India with ...,I understand you are asking about alternatives...
4,5,You provide smallholder farmers in India with ...,Marigold is an excellent trap crop that helps ...
5,6,You provide smallholder farmers in India with ...,"To prevent leaf blight in chili, start by usin..."
6,7,You provide smallholder farmers in India with ...,"Yes, you can maintain good grain production ev..."
7,8,You provide smallholder farmers in India with ...,Milk fever usually occurs within 72 hours afte...
8,9,You provide smallholder farmers in India with ...,"Yes, you can prepare nutritious animal feed at..."
9,10,You provide smallholder farmers in India with ...,Choosing the right cow breed depends on your s...


In [21]:
claude_3_7_df = get_llm_answers(qa_df, "anthropic/claude-3-7-sonnet-20250219", country, max_completion_tokens=25000, reasoning_effort="medium")

Saving results to results/AIEP_common_user_questions_India_Version-anthropic-claude-3-7-sonnet-20250219.csv


In [22]:
claude_3_7_df

,question_id,prompt,answer
0,1,You provide smallholder farmers in India with ...,# Factors Affecting Guntur Chilli Prices\n\nTh...
1,2,You provide smallholder farmers in India with ...,# Vaccination Schedule for Cows\n\nCalf vaccin...
2,3,You provide smallholder farmers in India with ...,# Preventing Mango Tree Rot and Fall\n\nTo pre...
3,4,You provide smallholder farmers in India with ...,# Alternatives to Cowdung as Fertilizer\n\nI u...
4,5,You provide smallholder farmers in India with ...,# Marigold as a Trap Crop: Benefits for Your F...
5,6,You provide smallholder farmers in India with ...,# Preventing Leaf Blight in Chili\n\nNamaste f...
6,7,You provide smallholder farmers in India with ...,# Increasing Grain in Animal Diets When Green ...
7,8,You provide smallholder farmers in India with ...,# Recognizing Milk Fever in Cows\n\nMilk fever...
8,9,You provide smallholder farmers in India with ...,"# Making Animal Feed at Home\n\nYes, you can d..."
9,10,You provide smallholder farmers in India with ...,# Choosing a Suitable Cow Breed for Your Farm\...


Deepseek models don't support the `reasoning_effort` parameter.

In [23]:
deepseek_v3_df = get_llm_answers(qa_df, "deepseek/deepseek-chat", country, max_completion_tokens=25000)

Saving results to results/AIEP_common_user_questions_India_Version-deepseek-deepseek-chat.csv


In [24]:
deepseek_v3_df

,question_id,prompt,answer
0,1,You provide smallholder farmers in India with ...,The price of Guntur Chilli depends on: \n\n1....
1,2,You provide smallholder farmers in India with ...,"For cows, the first vaccination (like for **Fo..."
2,3,You provide smallholder farmers in India with ...,**Answer:** To prevent mango tree rotting and ...
3,4,You provide smallholder farmers in India with ...,Cow dung is not a crop but an organic fertiliz...
4,5,You provide smallholder farmers in India with ...,Marigold acts as an excellent trap crop by att...
5,6,You provide smallholder farmers in India with ...,"To prevent leaf blight in chili, follow these ..."
6,7,You provide smallholder farmers in India with ...,"Yes, you can increase grain yield even without..."
7,8,You provide smallholder farmers in India with ...,**Answer:** Milk fever (hypocalcemia) occurs m...
8,9,You provide smallholder farmers in India with ...,"Yes, you can make high-quality animal feed at ..."
9,10,You provide smallholder farmers in India with ...,"To choose a suitable cow breed, consider these..."


In [25]:
deepseek_r1_df = get_llm_answers(qa_df, "deepseek/deepseek-reasoner", country, max_completion_tokens=25000)

Saving results to results/AIEP_common_user_questions_India_Version-deepseek-deepseek-reasoner.csv


In [26]:
deepseek_r1_df

,question_id,prompt,answer
0,1,You provide smallholder farmers in India with ...,Here are key factors affecting Guntur Chilli p...
1,2,You provide smallholder farmers in India with ...,The best vaccination age depends on the specif...
2,3,You provide smallholder farmers in India with ...,To prevent mango fruit rot and premature dropp...
3,4,You provide smallholder farmers in India with ...,Here are good alternatives to cowdung for fert...
4,5,You provide smallholder farmers in India with ...,Marigold acts as an excellent trap crop primar...
5,6,You provide smallholder farmers in India with ...,Here's how to prevent leaf blight in your chil...
6,7,You provide smallholder farmers in India with ...,"Yes, you can increase grain quantity even with..."
7,8,You provide smallholder farmers in India with ...,Here's how to recognize milk fever in your cow...
8,9,You provide smallholder farmers in India with ...,"Yes, absolutely! You can make good quality ani..."
9,10,You provide smallholder farmers in India with ...,"To choose the best cow breed for your farm, co..."


# Gemini

In [27]:
gemini_2_5_pro_df = get_llm_answers(qa_df, "gemini/gemini-2.5-pro-preview-05-06", country, max_completion_tokens=25000, reasoning_effort="medium")

Saving results to results/AIEP_common_user_questions_India_Version-gemini-gemini-2-5-pro-preview-05-06.csv


In [28]:
gemini_2_5_pro_df

,question_id,prompt,answer
0,1,You provide smallholder farmers in India with ...,Namaste! Several important things decide the p...
1,2,You provide smallholder farmers in India with ...,Namaste! Protecting your cows with vaccination...
2,3,You provide smallholder farmers in India with ...,Namaste! To prevent your mango tree from rotti...
3,4,You provide smallholder farmers in India with ...,Namaste! It seems there might be a small misun...
4,5,You provide smallholder farmers in India with ...,Namaste! Marigolds are very useful as a trap c...
5,6,You provide smallholder farmers in India with ...,"Namaste! To prevent leaf blight in your chili,..."
6,7,You provide smallholder farmers in India with ...,"Namaste! Yes, you can certainly increase your ..."
7,8,You provide smallholder farmers in India with ...,Namaste. Milk fever usually happens very close...
8,9,You provide smallholder farmers in India with ...,"Namaste! Yes, you can certainly make animal fe..."
9,10,You provide smallholder farmers in India with ...,Namaste! Choosing the right cow depends on you...


# Aggregate results

In [29]:
basename

'AIEP_common_user_questions_India_Version'

In [30]:
import glob

# Get a list of all CSV files in the results directory
csv_files = glob.glob(f"results/{basename}-*.csv")
# Print the list of CSV files
print(csv_files)

['results/AIEP_common_user_questions_India_Version-openai-o3-2025-04-16.csv', 'results/AIEP_common_user_questions_India_Version-gemini-gemini-2-5-pro-preview-05-06.csv', 'results/AIEP_common_user_questions_India_Version-deepseek-deepseek-reasoner.csv', 'results/AIEP_common_user_questions_India_Version-openai-o1-2024-12-17.csv', 'results/AIEP_common_user_questions_India_Version-anthropic-claude-3-7-sonnet-20250219.csv', 'results/AIEP_common_user_questions_India_Version-anthropic-claude-3-5-sonnet-20241022.csv', 'results/AIEP_common_user_questions_India_Version-openai-gpt-4o-2024-08-06.csv', 'results/AIEP_common_user_questions_India_Version-deepseek-deepseek-chat.csv', 'results/AIEP_common_user_questions_India_Version-openai-o4-mini-2025-04-16.csv']


In [31]:
len(csv_files)

9

In [32]:
# read all csv files, extract the model name from the filename, and concatenate them into a single dataframe
dfs = []
for file in csv_files:
    model_name = "-".join(os.path.basename(file).split("-")[1:]).split(".")[0]
    df = pd.read_csv(file)
    df["model"] = model_name
    dfs.append(df)

# concatenate all dataframes into a single dataframe
all_df = pd.concat(dfs, ignore_index=True)

# drop the prompt column
all_df = all_df.drop(columns=["prompt"])

# add country column
all_df["country"] = country

In [33]:
all_df

,question_id,answer,model,country
0,1,1. The first thing buyers look at is quality: ...,openai-o3-2025-04-16,India
1,2,1. No vaccine is needed at birth; just be sure...,openai-o3-2025-04-16,India
2,3,1. Keep rain-water from standing near the base...,openai-o3-2025-04-16,India
3,4,Cow-dung is only one kind of organic manure; i...,openai-o3-2025-04-16,India
4,5,Marigold works like a “sacrificial” plant that...,openai-o3-2025-04-16,India
...,...,...,...,...
373,38,In Bihar you can grow paddy in three main seas...,openai-o4-mini-2025-04-16,India
374,39,You can grow onions in three main seasons in I...,openai-o4-mini-2025-04-16,India
375,40,1. Choose an easy-to-grow variety such as oyst...,openai-o4-mini-2025-04-16,India
376,41,"1. Some wild mushrooms produce toxins that, if...",openai-o4-mini-2025-04-16,India


In [34]:
# add the questions by merging with the original dataframe
all_df = all_df.merge(qa_df[["question_id", "question"]], on="question_id", how="left")

In [35]:
all_df

,question_id,answer,model,country,question
0,1,1. The first thing buyers look at is quality: ...,openai-o3-2025-04-16,India,What factors affect the price of Guntur Chilli?
1,2,1. No vaccine is needed at birth; just be sure...,openai-o3-2025-04-16,India,What is the appropriate age for vaccination of...
2,3,1. Keep rain-water from standing near the base...,openai-o3-2025-04-16,India,What cheyyali to prevent the mango tree from r...
3,4,Cow-dung is only one kind of organic manure; i...,openai-o3-2025-04-16,India,What are alternative crops to cowdung?
4,5,Marigold works like a “sacrificial” plant that...,openai-o3-2025-04-16,India,What are the effects of marigold as a trap crop?
...,...,...,...,...,...
373,38,In Bihar you can grow paddy in three main seas...,openai-o4-mini-2025-04-16,India,When can you cultivate paddy in Bihar?
374,39,You can grow onions in three main seasons in I...,openai-o4-mini-2025-04-16,India,When can you cultivate onions?
375,40,1. Choose an easy-to-grow variety such as oyst...,openai-o4-mini-2025-04-16,India,How can I plant mushrooms?
376,41,"1. Some wild mushrooms produce toxins that, if...",openai-o4-mini-2025-04-16,India,What diseases are caused by mushrooms?


In [36]:
# add a response_id column
all_df["response_id"] = all_df.index + 1

In [37]:
all_df

,question_id,answer,model,country,question,response_id
0,1,1. The first thing buyers look at is quality: ...,openai-o3-2025-04-16,India,What factors affect the price of Guntur Chilli?,1
1,2,1. No vaccine is needed at birth; just be sure...,openai-o3-2025-04-16,India,What is the appropriate age for vaccination of...,2
2,3,1. Keep rain-water from standing near the base...,openai-o3-2025-04-16,India,What cheyyali to prevent the mango tree from r...,3
3,4,Cow-dung is only one kind of organic manure; i...,openai-o3-2025-04-16,India,What are alternative crops to cowdung?,4
4,5,Marigold works like a “sacrificial” plant that...,openai-o3-2025-04-16,India,What are the effects of marigold as a trap crop?,5
...,...,...,...,...,...,...
373,38,In Bihar you can grow paddy in three main seas...,openai-o4-mini-2025-04-16,India,When can you cultivate paddy in Bihar?,374
374,39,You can grow onions in three main seasons in I...,openai-o4-mini-2025-04-16,India,When can you cultivate onions?,375
375,40,1. Choose an easy-to-grow variety such as oyst...,openai-o4-mini-2025-04-16,India,How can I plant mushrooms?,376
376,41,"1. Some wild mushrooms produce toxins that, if...",openai-o4-mini-2025-04-16,India,What diseases are caused by mushrooms?,377


In [38]:
# shuffle the dataframe
all_df = all_df.sample(frac=1, random_state=1337).reset_index(drop=True)
all_df

,question_id,answer,model,country,question,response_id
0,4,Cow dung is not a crop but an organic fertiliz...,deepseek-deepseek-chat,India,What are alternative crops to cowdung?,298
1,33,Namaste! To manage pests and diseases in your ...,gemini-gemini-2-5-pro-preview-05-06,India,What are some organic methods of controlling p...,75
2,31,1. First separate the lame animal from the res...,openai-o3-2025-04-16,India,How to treat lameness in animals?,31
3,24,To manage water in rice cultivation: \n\n1. *...,deepseek-deepseek-chat,India,How do I manage water in rice cultivation?,318
4,21,Namaste! The amount of feed your cow needs dep...,gemini-gemini-2-5-pro-preview-05-06,India,How much animal feed should be given to milk c...,63
...,...,...,...,...,...,...
373,42,There are many types of mushrooms that farmers...,openai-o1-2024-12-17,India,What types of mushrooms are there?,168
374,23,"During pregnancy, your cow needs proper nutrit...",anthropic-claude-3-5-sonnet-20241022,India,What should be fed to a pregnant cow?,233
375,13,1. In 2022–23 India produced over 220 million ...,openai-o4-mini-2025-04-16,India,What is the status of milk production in India?,349
376,22,# Feed Requirements for Rural Cows\n\nFor a ty...,anthropic-claude-3-7-sonnet-20250219,India,How much feed is needed for a rural cow?,190


In [39]:
# reorder the columns
all_df = all_df[["model", "question_id", "response_id", "question", "answer", "country"]]
all_df

,model,question_id,response_id,question,answer,country
0,deepseek-deepseek-chat,4,298,What are alternative crops to cowdung?,Cow dung is not a crop but an organic fertiliz...,India
1,gemini-gemini-2-5-pro-preview-05-06,33,75,What are some organic methods of controlling p...,Namaste! To manage pests and diseases in your ...,India
2,openai-o3-2025-04-16,31,31,How to treat lameness in animals?,1. First separate the lame animal from the res...,India
3,deepseek-deepseek-chat,24,318,How do I manage water in rice cultivation?,To manage water in rice cultivation: \n\n1. *...,India
4,gemini-gemini-2-5-pro-preview-05-06,21,63,How much animal feed should be given to milk c...,Namaste! The amount of feed your cow needs dep...,India
...,...,...,...,...,...,...
373,openai-o1-2024-12-17,42,168,What types of mushrooms are there?,There are many types of mushrooms that farmers...,India
374,anthropic-claude-3-5-sonnet-20241022,23,233,What should be fed to a pregnant cow?,"During pregnancy, your cow needs proper nutrit...",India
375,openai-o4-mini-2025-04-16,13,349,What is the status of milk production in India?,1. In 2022–23 India produced over 220 million ...,India
376,anthropic-claude-3-7-sonnet-20250219,22,190,How much feed is needed for a rural cow?,# Feed Requirements for Rural Cows\n\nFor a ty...,India


In [40]:
# save the dataframe to a CSV file
all_df.to_csv(f"{basename}-all.csv", index=False)